In [13]:
import pandas as pd
import numpy as np
import pickle

In [14]:
df = pd.read_csv("settingData.csv", encoding='cp949')

In [15]:
df.drop(["Unnamed: 0","TM","현황","VIS","PS"],axis=1, inplace=True)

In [16]:
df['delayTime'] = df['delayTime'].fillna(500)

In [17]:
#one-hot-encoding
df = pd.get_dummies(df, columns=['항공사',"ORIGIN","DEST","weekend"])

In [18]:
temp = df.delayTime.value_counts()
l = []
for i in temp.keys():
    if temp[i] == 1:
        l.append(i)
lb = [df[df.delayTime==i].index for i in l]
for i in lb:
    df = df.drop(i)

In [19]:
%matplotlib inline
import matplotlib.pyplot as plt

In [20]:
df = df[df['delayTime']>=0]
df = df[df['delayTime']<=50]

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 105049 entries, 0 to 143927
Data columns (total 28 columns):
CA_TOT         105049 non-null int64
Date           105049 non-null int64
PA             105049 non-null int64
TD             105049 non-null int64
TMP            105049 non-null int64
Time           105049 non-null int64
WD             105049 non-null int64
WSPD           105049 non-null int64
delayTime      105049 non-null float64
항공사_대한항공       105049 non-null uint8
항공사_아시아나항공     105049 non-null uint8
항공사_에어부산       105049 non-null uint8
항공사_에어필립       105049 non-null uint8
항공사_이스타항공      105049 non-null uint8
항공사_제주항공       105049 non-null uint8
항공사_진에어        105049 non-null uint8
항공사_티웨이항공      105049 non-null uint8
ORIGIN_김포공항    105049 non-null uint8
ORIGIN_제주공항    105049 non-null uint8
DEST_김포공항      105049 non-null uint8
DEST_제주공항      105049 non-null uint8
weekend_FRI    105049 non-null uint8
weekend_MON    105049 non-null uint8
weekend_SAT    105049 non-null uint8

In [22]:
#테스트 세트 생성
def split_flight_test(data, test_ratio):
    shuffled_indices = np.random.permutation(len(data))
    test_set_size = int(len(data) * test_ratio)
    test_indices = shuffled_indices[:test_set_size]
    flight_indices = shuffled_indices[test_set_size:]
    return data.iloc[flight_indices], data.iloc[test_indices]

In [23]:
# 무작위로 샘플 선택하여 데이터셋의 20% 분리
flight_set, test_set = split_flight_test(df, 0.2)
print(len(flight_set), "flight +", len(test_set), "test")

84040 flight + 21009 test


In [24]:
from sklearn.model_selection import train_test_split
flight_set, test_set = train_test_split(df, test_size=0.2, random_state=42)

In [25]:
#계층별로 데이터셋에 충분한 샘플 수가 있어야함 -> 중요도 편향 방지
df["delay"] = df['delayTime']

In [26]:
#계층별 샘플링
from sklearn.model_selection import StratifiedShuffleSplit
 
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for flight_index, test_index in split.split(df, df["delay"]):
    strat_flight_set = df.loc[flight_index]
    strat_test_set = df.loc[test_index]

c:\users\kss09\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  
c:\users\kss09\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  import sys


In [27]:
#delay특성 삭제 데이터 원래 상태로 복구
for set_ in (strat_flight_set, strat_test_set):
    set_.drop("delay", axis=1, inplace=True)

In [28]:
#훈련데이터셋 손상 방지 복사본 만들어 사용
data = strat_flight_set.copy()

In [29]:
data_labels = strat_flight_set["delayTime"].copy()
data = strat_flight_set.drop("delayTime",axis=1)

In [30]:
data.dropna(axis=0, inplace=True)
data_labels.dropna(axis=0, inplace=True)

In [31]:
from sklearn.ensemble import RandomForestRegressor
forest_reg = RandomForestRegressor()
forest_reg.fit(data, data_labels)

c:\users\kss09\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [32]:
saved_model = pickle.dumps(forest_reg)

In [33]:
from sklearn.externals import joblib

joblib.dump(forest_reg, 'delayTime.pkl')

c:\users\kss09\appdata\local\programs\python\python37\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


['delayTime.pkl']